# ISO TC/211 Harmonised Model Controls

Connect to the EA app and model repository

In [ ]:
from Parameters import *
from EAConnect import *
from HM_Controls import *
import sys
import pandas as pd

# Open EA Repository and find Model
eaApp = openEAapp()
eaRepo = openEArepo(eaApp,repo_path)
try:
    omMod = eaRepo.Models.GetByName(modelName)
    printTS('Model "' + modelName + '" found with PackageGUID ' + omMod.PackageGUID )
except Exception as e:
    printTS('Model  "' + modelName + '" not found!')
    closeEA(eaRepo)
    sys.exit()
printTS('Number of main packages: ' + str(omMod.Packages.Count))
timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

## Select a package in EA before continuing

In [ ]:
thePackage = eaRepo.GetTreeSelectedPackage()
printTS('Selected package name: ' + thePackage.Name)

**Package tags and alias**

Add or update package tags and alias

In [ ]:
lstMd = {}
#lstMd['name'] = 'Training Data Markup Language for Artificial Intelligence'
lstMd['name'] = 'Training Data Markup Language for Artificial Intelligence - Part 1: Conceptual Model Standard'
lstMd['number'] = '19178-1'
lstMd['edition'] = '1'
lstMd['publicationDate'] = ''
lstMd['yearVersion']= '2024'

eaEl = thePackage.Element

#Update existing tags and add missing tags
for key, value in lstMd.items():
    print(f"{key}: {value}")
    try:
        eaTag = eaEl.TaggedValues.GetByName(key)
        if not eaTag is None:
            eaTag.Value = value
            eaTag.Update()
        else:
            print('New tag!')
            eaTag = eaEl.TaggedValues.AddNew(key,value)
            eaTag.Update()    
    except:
        print('Something went wrong!')
eaEl.TaggedValues.Refresh()

#Set alias
eaEl.Alias = lstMd['name']
eaEl.Update()

#TODO:Update status?



**Duplicates:**

Check that there are no duplicate element names

In [ ]:
df = duplicateElements(thePackage)
non_unique = df[df.duplicated(subset=['ElementName'], keep=False)]
errCount = len(non_unique)
printTS('Number of errors: ' + str(errCount))

if errCount > 0:
    print('')
    print('Duplicate elements: ')
    for index, row in non_unique.iterrows():
        combined_string = f"{row['ElementName']}"
        print(combined_string)

**Data type references:**

Check that all data types are references to an element.
Missing references to primitive data types are fixed in the script.

The results are stored in a data frame for further use.

In [27]:
#df = pd.DataFrame(columns=['FullPath','Package','Element','Property','DependentPackage','DependentElement','GUID'])
df = listClassifiers(eaRepo,thePackage)
noRef = df[df['GUID'].isna()]
errCount = len(noRef)
printTS('')
printTS('Number of errors: ' + str(errCount))

print('')
file_path = mainFolder + '\\Controls.xlsx'
# Export to Excel 
writer = pd.ExcelWriter(file_path)
df.to_excel(writer,'Datatypes') 
print(f"Exported file: {file_path}") 
writer.close()

print('')
print('Attributes without reference:')
for index, row in noRef.iterrows():
    combined_string = f"{row['Package']}.{row['Element']}.{row['Property']} (Data type:{row['DependentElement']})"
    print(combined_string)




2024-04-25 18:36:30   
2024-04-25 18:36:30   Number of errors: 0

Exported file: C:\Data\GitHub\ISO TC211\HMMG\EditorialVersion\Controls.xlsx

Attributes without reference:


C:\Users\JETKNU\AppData\Local\Temp\ipykernel_19112\92195387.py:12: FutureWarning: Starting with pandas version 3.0 all arguments of to_excel except for the argument 'excel_writer' will be keyword-only.
  df.to_excel(file_path,'Datatypes')


**Dependencies:**

Check which packages data types and relations are connected to.

In [ ]:
dfCounts = df.groupby(['DependentPackage']).size()
print(dfCounts)

**Dependencies:**

List elements from a specific package 

In [ ]:
strFilter = 'ISO TC211.'

df_cleaned = df.dropna(subset=['DependentPackage'])
filtered_df = df_cleaned[df_cleaned['DependentPackage'].str.startswith(strFilter)]

if len(filtered_df) > 0:
    print('Refered elements from package "' + strFilter + '*"')
    for index, row in filtered_df.iterrows():
        print(f"{row['Element']}.{row['Property']} (Data type:{row['DependentPackage']}.{row['DependentElement']})")
else:
    print('No elements found!')    

**Definitions:**

Check that all classes, attributes and navigable associations have definitions

In [28]:
#defDf = pd.DataFrame(columns=['Type','PackageName','ElementName','PropertyName','Supplier'])
defDf = listMissingDefinitions(eaRepo,thePackage)

errCount = len(defDf)
printTS('')
printTS('Number of errors: ' + str(errCount))
print('')
# Export to Excel 
#writer = pd.ExcelWriter(file_path)
defDf.to_excel(writer,'Definitions') 
print(f"Exported file: {file_path}") 
writer.close()

noDef = defDf[defDf['PropertyName'].isna() & defDf['Supplier'].isna()]
if len(noDef) > 0:
    print('')
    print('Elements without definitions (' + str(len(noDef)) + '):')
    for index, row in noDef.iterrows():
        combined_string = f"{row['PackageName']}.{row['ElementName']}"
        print(combined_string)

noDef = defDf[defDf['Type']=='Attribute']
if len(noDef) > 0:
    print('')
    print('Attributes without definitions (' + str(len(noDef)) + '):')
    for index, row in noDef.iterrows():
        combined_string = f"{row['PackageName']}.{row['ElementName']}.{row['PropertyName']}"
        print(combined_string)

noDef = defDf[defDf['Type']=='Code value']
if len(noDef) > 0:
    print('')
    print('Code values without definitions (' + str(len(noDef)) + '):')
    for index, row in noDef.iterrows():
        combined_string = f"{row['PackageName']}.{row['ElementName']}.{row['PropertyName']}"
        print(combined_string)

noDef = defDf[defDf['Type']=='Role name']
if len(noDef) > 0:
    print('')
    print('Navigable association ends without role name (' + str(len(noDef)) + '):')
    for index, row in noDef.iterrows():
        combined_string = f"{row['PackageName']}.{row['ElementName']} towards {row['Supplier']}"
        print(combined_string)

noDef = defDf[defDf['Type']=='Role']
if len(noDef) > 0:
    print('')
    print('Navigable association ends without definition (' + str(len(noDef)) + '):')
    for index, row in noDef.iterrows():
        if row['PropertyName'] != None:
            combined_string = f"{row['PackageName']}.{row['ElementName']}.{row['PropertyName']} towards {row['Supplier']}"
        else:
            combined_string = f"{row['PackageName']}.{row['ElementName']} towards {row['Supplier']}"
        print(combined_string)


2024-04-25 18:37:49   ERROR|Missing definition for:Class in package ISO 19178-1 Edition 1:AI_AbstractLabel
2024-04-25 18:37:49   ERROR|Missing definition for attribute in package ISO 19178-1 Edition 1: AI_AbstractLabel.isNegative
2024-04-25 18:37:49   ERROR|Missing definition for attribute in package ISO 19178-1 Edition 1: AI_AbstractLabel.confidence
2024-04-25 18:37:49   ERROR|Missing definition for:Class in package ISO 19178-1 Edition 1:AI_AbstractTask
2024-04-25 18:37:49   ERROR|Missing definition for attribute in package ISO 19178-1 Edition 1: AI_AbstractTask.id
2024-04-25 18:37:49   ERROR|Missing definition for attribute in package ISO 19178-1 Edition 1: AI_AbstractTask.datasetId
2024-04-25 18:37:49   ERROR|Missing definition for attribute in package ISO 19178-1 Edition 1: AI_AbstractTask.description
2024-04-25 18:37:49   ERROR|Missing definition for:Class in package ISO 19178-1 Edition 1:AI_AbstractTrainingData
2024-04-25 18:37:49   ERROR|Missing definition for attribute in packa

C:\Users\JETKNU\AppData\Local\Temp\ipykernel_19112\1931451279.py:10: FutureWarning: Starting with pandas version 3.0 all arguments of to_excel except for the argument 'excel_writer' will be keyword-only.
  defDf.to_excel(writer,'Definitions')


**Diagram representation:**

Check that all elements are shown in at least one diagram

In [ ]:
dfE = elementsInDiagrams(thePackage)
errCount = len(dfE)
printTS('')
printTS('Number of errors: ' + str(errCount))

if errCount > 0:
    print('')
    print('Elements that are not in any diagram:')
    for index, row in dfE.iterrows():
        combined_string = f"{row['PackageName']}.{row['ElementName']}"
        print(combined_string)    


**Diagram layout:**

Setting diagram fonts to Cambria and hiding "isSubstitutable" labels

In [ ]:
recDiagramCleaning(thePackage)

**Export to XMI:**

Export the package to XMI for upload to GitHub

In [ ]:
# Replace colons (":") with underscores ("_") in the filename
fName = thePackage.Name.replace(":", "_")
# Replace forward slashes ("/") with an empty string
fName = fName.replace("/", "")
# Combine the modified filename with the path and add the ".xml" extension
fName = xmiPath + fName + ".xml"

thePackage.IsControlled = -1
thePackage.XMLPath = fName
thePackage.BatchSave = 1
thePackage.BatchLoad = 1
thePackage.Update

#XmiExportType = 3
pI = eaRepo.GetProjectInterface()
result = pI.ExportPackageXMI(thePackage.PackageGUID, 3, 1, -1, 1, 0, fName)
print(result)